# 모델 저장하기 & 불러오기
- 3가지 핵심함수
    1. `torch.save` : 직렬화된 객체를 디스크에 저장
        - Python의 pickle(리스트나 클래스같은 자료형을 저장/로드)을 사용하여 직렬화
        - 모든 종류의 객체의 모델, Tensor 및 사전을 저장
    2. 'torch.load:pickle`을 사용하여 저장된 객체 파일들을 역직렬화하여 메모리에 올림
    3. `torch.nn.Module.load_state_dict` : state_dict 사용하여 모델의 매겨변수를 불러움

## state_dict란
> 한마디로 model layer KEY tensor VALUE의 dictionary

- Pytorch에서 `torch.nn.Module` 모델의 학습 가능한 매개변수들은 모델의 매개변수에 포함되어 있음
    - `model.parameter()`로 접근
- 간단하게 말해 각 계층을 매개변수 텐서로 매핑되는 Python dictionary 객체
    - 학습 가능한 매개변수를 갖는 계층(합성곱 계층, 선형 계층 등) 및 등록된 버퍼들만이 모델의 state_dict에 항목을 가짐
    - torch.optim또한 옵티마이저의 상태 뿐만 아니라 사용된 하이퍼 매개변수 정보를 포함된 state_dict를 가짐
    

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = TheModelClass()

# init optim
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# model의 state_dict
print(f'Model state_dict :')
for param_tensor in model.state_dict():
    print(f'{param_tensor}\t{model.state_dict()[param_tensor].size()}')
    
# optim의 state_dict 출력
print('Optimizer state_dict :')
for var_name in optimizer.state_dict():
    print(f'{var_name}\t{optimizer.state_dict()[var_name]}')
        

Model state_dict :
conv1.weight	torch.Size([6, 3, 5, 5])
conv1.bias	torch.Size([6])
conv2.weight	torch.Size([16, 6, 5, 5])
conv2.bias	torch.Size([16])
fc1.weight	torch.Size([120, 400])
fc1.bias	torch.Size([120])
fc2.weight	torch.Size([84, 120])
fc2.bias	torch.Size([84])
fc3.weight	torch.Size([10, 84])
fc3.bias	torch.Size([10])
Optimizer state_dict :
state	{}
param_groups	[{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [2883275561504, 2883275721032, 2883275721320, 2883275721392, 2883275721464, 2883275721536, 2883275721608, 2883275721680, 2883275721752, 2883275721824]}]


## 추론(interface)를 위해 모델 저장/로드
- state_dict 저장/로드
- `pt`나 `pth` 확장자 주로 사용

In [11]:
# save
PATH = './data/model_weight/test_sd.pt'
torch.save(model.state_dict(), PATH)

In [7]:
# load
model = TheModelClass()
model.load_state_dict(torch.load(PATH))
model.eval() # 드롭아웃 및 배치 정규화를 평가 모드로 설정

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

## 전체 모델 저장/불러오기
- Python의 `pickle`모듈을 사용
    - 단점 : 모델 그 자체를 저장하지 않기 떄문에 직렬화된 데이터가 모델을 저장할 때 특정 클래스/디렉터리 경로에 얽매임
        - 그래서 리펙토링 후 혹은 다른 프로젝트에서 만든 모델 사용하면 안될 수 있음

In [13]:
# save
torch.save(model, PATH)

C:\ProgramData\Anaconda3\envs\pytch_env\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type TheModelClass. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [14]:
# load
model = torch.load(PATH)
model.eval()

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

## 추론 / 학습 재개를 위한 일반 체크포인트 저장/불러오기
- 반드시 model의 state_dict보다 많은 것들을 저장해야함
- 모델이 학습을 하며 갱신되는 버퍼와 매개변수가 포함된 optimizer의 state_dict도 함께 저장
    - 그 외에도 마지막 epoch, 최근에 기록된 학습 손실, 외부 torch.nn.Embedding 계층 등도 함께 저장
- 여러가지를 저장하려면 dictionary 자료형으로 만든 후 `torch.save()`를 사용해 직렬화
    - 이러한 체크포인트를 저장할 때는 `.tar`확장자를 사용하는 것이 일반적
- 항목들을 불러올 때에는 먼저 모델과 옵티마이저를 초기화한 후 `torch.load()`를 사용하여 사전을 불러옴
- 추론 실행전에는 반드시 `model.eval()`을 호출하여 드롭아웃, 배치 정규화를 평가모드로 설정
    - 학습을 계속 하려면 `model.train()`로 전환

In [16]:
# save
epoch = 10
loss = 0.0
torch.save({'epoch' : epoch,
            'model_state_dict' : model.state_dict(),
            'optimizer_state_dict' : optimizer.state_dict(),
            'loss' : loss,
           }, PATH)

In [17]:
# load
model = TheModelClass()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

## 여러개의 모델을 하나의 파일에 저장

In [20]:
# save
modelA = TheModelClass()
modelB = TheModelClass()
optimizerA = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizerB = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

torch.save({
            'modelA_state_dict' : modelA.state_dict(),
            'modelB_state_dict' : modelB.state_dict(),
            'optimizerA_state_dict' : optimizerA.state_dict(),
            'optimizerB_state_dict' : optimizerB.state_dict(),
            }, PATH)


In [27]:
# load
modelA = TheModelClass()
modelB = TheModelClass()
optimizerA = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizerB = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA = checkpoint['optimizerA_state_dict']
optimizerB = checkpoint['optimizerB_state_dict']

modelA.eval()
modelB.eval()

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

## 다른 모델의 매개변수를 사용하여 빠르게 모델 시작하기
- 몇몇 key를 제외하고 state_dict의 일부를 불러오거나 적재하려는 모델보다 더 많은 키를 갖고 있는 state_dict를 불러올 때
     - `load_state_dict()`함수에서 `strict`인자를 False로 설정
             - 일치하지 않는 키들을 무시

In [28]:
# save
torch.save(modelA.state_dict(), PATH)

In [30]:
# load
modelB = TheModelClass()
modelB.load_state_dict(torch.load(PATH), strict=False)

<All keys matched successfully>

## 다른 device설정으로 model 불러올 때 

In [33]:
# GPU로 저장하고 CPU로 불러올 때
# save
torch.save(model.state_dict(), PATH)
# load
device = torch.device('cpu')
model = TheModelClass()
# map_location 인자를 사용해 CPU 장치에 동적으로 재배치
model.load_state_dict(torch.load(PATH, map_location=device))


# GPU로 저장하고 GPU로 불러올 때 
# save
torch.save(model.state_dict(), PATH)
# load
device = torch.device('cuda')
model = TheModelClass()
model.load_state_dict(torch.load(PATH))
# CUDA에 최적화된 모델로 변환
model.to(device)
# model에서 사용하는 input들은 input.to(device)를 호출해줘야 함 -> 복사본 리턴하니까
# input = input.to(device)로 받아줘야 함


# CPU로 저장하고 GPU로 불러올 때 
# save
torch.save(model.state_dict(), PATH)
# load
device = torch.device('cuda')
model = TheModelClass()
# map_location 인자를 사용해 GPU 장치에 동적으로 재배치
model.load_state_dict(torch.load(PATH, map_location='cuda:0'))
model.to(device)
# model에서 사용하는 input들은 input.to(device)를 호출해줘야 함 -> 복사본 리턴하니까
# input = input.to(device)로 받아줘야 함

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)